In [1]:
import numpy as np
import scipy as sp
import sys
sys.path.append('C:/Users/rannu/OneDrive - NTNU/Desktop/VsPython/Spain/'+\
                'NMfDS/Assignments/Ass4')
from RTBP_definitions import r1, r2, OMEGA, ODE_R3BP, Jacobi_first_integral

### Part A)

In [2]:
def ODE_solver(func, x0, t_max, eval_pts, tol=1e-12, t_min=0, method='DOP853', hamiltonian=0):
    # Variable explanation:
    # func: the function f in the system of ODEs
    # hamiltonian: the hamiltonian of the system
    # x0: the initial condition, a vector of size n
    # t_max: the maximum time
    # eval_pts: number of points at which the solution is evaluated
    # tol: tolerance for checking if the hamiltonian is constant
    # t_min: the minimum time
    # method: the method used to solve the system of ODEs

    # RETURNS: the solution of the system of ODEs as a scipy.integrate.solve_ivp object

    t_eval = np.linspace(t_min, t_max, eval_pts)

    sol = sp.integrate.solve_ivp(
        func, [t_min, t_max], x0, method=method, t_eval=t_eval, atol=1e-16, rtol=1e-18)

    # Check if the hamiltonian remains constant for all points, by checking the error. break iff error > tol
    # Also chek the determinant of the matrix [[x3, x4], [x5, x6]]
    if hamiltonian == 0:
        print("Warning: No hamiltonian given. The hamiltonian will not be checked for constancy.")
    else:
        for i in range(len(sol.t)):
            if abs(hamiltonian(sol.y[:, i]) - hamiltonian(x0)) > tol:
                print("Hamiltonian is not constant for all points. Error = ", abs(
                    hamiltonian(sol.y[:, i]) - hamiltonian(x0)))
                print("Hamiltonian at x0 = ", hamiltonian(x0))
                print("Hamiltonian at x = ", hamiltonian(sol.y[:, i]))
                break
    return sol

In [3]:
X0 = np.array([1.119748482176185, 0,0.,-0.2260036463376957], dtype=np.float64)
t_max = np.float64(6.219656012174353)
eval_pts = 1000 # Integer
mu =np.float64(0.012)



sol = ODE_solver(lambda t, X: ODE_R3BP(t, mu, X), X0, t_max, eval_pts, 
                 hamiltonian=lambda X: Jacobi_first_integral(mu, X[0], X[1], 
                                                             X[2], X[3]))

# Final values of the solution
print("Final values of the solution:")
print("x = ", sol.y[:, -1])

Final values of the solution:
x =  [ 1.11974848e+00  5.06660686e-13  2.31125882e-13 -2.26003646e-01]


c:\Users\rannu\OneDrive - NTNU\Desktop\VsPython\ntnusession\lib\site-packages\scipy\integrate\_ivp\common.py:47: UserWarning: At least one element of `rtol` is too small. Setting `rtol = np.maximum(rtol, 2.220446049250313e-14)`.
  warn("At least one element of `rtol` is too small. "


In [4]:
# Check if it also works backwards in time
t_max = np.float64(-6.219656012174353)
sol = ODE_solver(lambda t, X: ODE_R3BP(t, mu, X), X0, t_max, eval_pts, 
                 hamiltonian=lambda X: Jacobi_first_integral(mu, X[0], X[1], 
                                                             X[2], X[3]))

# Final values of the solution
print("Final values of the solution:")
print("x = ", sol.y[:, -1])

Final values of the solution:
x =  [ 1.11974848e+00 -5.06660686e-13 -2.31125882e-13 -2.26003646e-01]


In [5]:
#Hamiltonian system of ODE associated to the Hamitonian function H(x, y, px, py)
def ODE_Hamiltonian(mu, x1, x2, y1, y2):
    return (y1**2 + y2**2) / 2 + x2 * y1 - x1 * y2 \
            - (1 - mu) / r1(mu, x1, x2) - mu / r2(mu, x1, x2)

px = lambda X: X[2] - X[1]
py = lambda X: X[3] - X[0]

sol = ODE_solver(lambda t, X: ODE_R3BP(t, mu, X), X0, t_max, eval_pts, 
                 hamiltonian=lambda X: ODE_Hamiltonian(mu, X[0], X[1], X[2] \
                                                       - X[1], X[3] - X[0]))

# Final values of the solution
print("Final values of the solution:")
print("x = ", sol.y[:, -1])

Hamiltonian is not constant for all points. Error =  2.2884417623547648e-05
Hamiltonian at x0 =  1.5148358065775753
Hamiltonian at x =  1.5148129221599518
Final values of the solution:
x =  [ 1.11974848e+00 -5.06660686e-13 -2.31125882e-13 -2.26003646e-01]


Assignment 6 finished.

### Now, JIT!

In [6]:
sys.path.append('C:/Users/rannu/OneDrive - NTNU/Desktop/VsPython/Spain/\
                NMfDS/Assignments/Ass4')
from RTBP_definitions import r1, r2, OMEGA, ODE_R3BP, Jacobi_first_integral
import RTBP_definitions_JIT as RTBP_JIT
# For the JIT compilation:
import numba as nb
# "freeze" some portion of a function's arguments and/or keywords 
# resulting in a new object with a simplified signature:
# from functools import partial

In [7]:
RTBP_JIT.hi()

hia


In [8]:
@nb.jit
def r1_JIT(mu, x, y):
    return np.sqrt((x - mu)**2 + y**2)


@nb.jit
def r2_JIT(mu, x, y):
    return np.sqrt((x-mu+1)**2 + y**2)


@nb.jit  # Set "nopython" mode for best performance, equivalent to @njit
def OMEGA_JIT(mu, x, y):
    return 0.5 * (x**2 + y**2) + (1 - mu) / r1(mu, x, y) + mu / r2(mu, x, y) \
        + 0.5 * (1 - mu) * mu


@nb.jit  # Set "nopython" mode for best performance, equivalent to @njit
def ODE_R3BP_JIT(t, X, mu):
    # ODEs of the restricted three body problem
    return [X[2], X[3], 2*X[3] + X[0] - (1 - mu)*(X[0] - mu) /
            r1(mu, X[0], X[1])**3 - mu * (X[0] - mu + 1) /
            r2(mu, X[0], X[1])**3, -2*X[2] + X[1] - (1 - mu) * X[1]
            / r1(mu, X[0], X[1])**3 - mu * X[1] / r2(mu, X[0], X[1])**3]


@nb.jit  # Set "nopython" mode for best performance, equivalent to @njit
def Jacobi_first_integral_JIT(X, mu):
    x = X[0]
    y = X[1]
    vx = X[2]
    vy = X[3]
    # Function to compute the Jacobi first integral
    # This should be constant for a given value of mu
    return 2*OMEGA(mu, x, y) - (vx**2 + vy**2)

In [9]:
# # Define ODE function for R3BP using the global variable
# @nb.jit
# def ODE_R3BP_with_mu(t, X):
#     return ODE_R3BP_JIT(t, mu, X)

# # Define Hamiltonian function using the global variable
# @nb.jit
# def Hamiltonian_with_mu(X):
#     return Jacobi_first_integral_JIT(mu, X[0], X[1], X[2], X[3])


In [14]:
#@nb.jit(nopython=True)
def ODE_solver_JIT(func, x0, t_max, eval_pts, mu, tol=1e-12, t_min=0, method='DOP853', hamiltonian=0):
    # Variable explanation:
    # func: the function f in the system of ODEs
    # hamiltonian: the hamiltonian of the system
    # x0: the initial condition, a vector of size n
    # t_max: the maximum time
    # eval_pts: number of points at which the solution is evaluated
    # tol: tolerance for checking if the hamiltonian is constant
    # t_min: the minimum time
    # method: the method used to solve the system of ODEs

    # RETURNS: the solution of the system of ODEs as a scipy.integrate.solve_ivp object

    t_eval = np.linspace(t_min, t_max, eval_pts, dtype=np.float64)

    sol = sp.integrate.solve_ivp(
        func, [t_min, t_max], x0, method=method, t_eval=t_eval, atol=1e-16, rtol=1e-18)

    # Check if the hamiltonian remains constant for all points, by checking the error. break iff error > tol
    # Also chek the determinant of the matrix [[x3, x4], [x5, x6]]
    if hamiltonian == 0:
        print("Warning: No hamiltonian given. The hamiltonian will not be checked for constancy.")
    else:
        for i in range(len(sol.t)):
            if abs(hamiltonian(sol.y[:, i]) - hamiltonian(x0)) > tol:
                print("Hamiltonian is not constant for all points. Error = ", abs(
                    hamiltonian(sol.y[:, i]) - hamiltonian(x0)))
                print("Hamiltonian at x0 = ", hamiltonian(x0))
                print("Hamiltonian at x = ", hamiltonian(sol.y[:, i]))
                break
    return sol

In [15]:
#print(ODE_solver_JIT.signatures)

# sol = ODE_solver_JIT(RTBP_JIT.ODE_R3BP, X0, t_max, eval_pts, 
#                  hamiltonian=RTBP_JIT.Jacobi_first_integral)
sol = ODE_solver_JIT(ODE_R3BP_JIT, X0, t_max, eval_pts, mu,
                 hamiltonian=0)
#print(ODE_solver_JIT.inspect_types())
print("Final values of the solution:")
print("x = ", sol.y[:, -1])

c:\Users\rannu\OneDrive - NTNU\Desktop\VsPython\ntnusession\lib\site-packages\scipy\integrate\_ivp\common.py:47: UserWarning: At least one element of `rtol` is too small. Setting `rtol = np.maximum(rtol, 2.220446049250313e-14)`.
  warn("At least one element of `rtol` is too small. "


TypeError: not enough arguments: expected 3, got 2

In [ ]:
import numba as nb

@nb.njit
def foo():
    acc = 0
    for i in range(100):
        acc += i
    return acc

print(foo.signatures)  # Output: []
foo()  # Call the jitted function
print(foo.signatures)  # Output: [()]

[]
[()]
